# Data Streaming 1: Running a Tick Data Server

# Contents

## 1. Running a Tick Data Server
    - Generate data with geometric brownian motion
    - Create a socket using ZeroMQ and publish data to socket

# See other notebook for the following
## 2. Connecting a Tick Data Client

## 3. Real Time Signal Generation 

## 4. Plotting Streaming Data

In [1]:
import zmq
import math
import time
import random

# 1. Running a Tick Data Server

In [2]:
context = zmq.Context()
socket = context.socket(zmq.PUB)
socket.bind('tcp://0.0.0.0:5555')

<SocketContext(bind='tcp://0.0.0.0:5555')>

In [3]:
class InstrumentPrice(object): 
    def __init__(self):
        self.symbol = 'SYMBOL'
        self.t = time.time()
        self.value = 100.
        self.sigma = 0.4
        self.r = 0.01
    def simulate_value(self):
        ''' Generates a new, random stock price. '''
        t = time.time()
        dt=(t-self.t)/(252*8*60*60)
        dt *= 500
        self.t = t
        self.value *= math.exp((self.r - 0.5 * self.sigma ** 2) * dt + self.sigma * math.sqrt(dt) * random.gauss(0, 1))
        return self.value

In [4]:
ip = InstrumentPrice()
while True:
    msg = '{} {:.2f}'.format(ip.symbol, ip.simulate_value()) 
    print(msg, end='\r')
    socket.send_string(msg)
    time.sleep(random.random() * 2)

KeyboardInterrupt: 